# Modely by Topic

Here an attempt is made to fit a model for one topic and test it on the 4 other topics. This can reveal similarities between text from different topics.

The hyperparameters will not be optimized again, as this is not the main part of the research, but will be taken from previous optimization.

##  Load train and test data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Provide the correct file path
file_path_train = "/content/drive/My Drive/BA THESIS/data/train_NB_lem.csv"
#file_path_val = "/content/drive/My Drive/BA THESIS/data/val_NB_lem.csv"
file_path_test = "/content/drive/My Drive/BA THESIS/data/test_NB_lem.csv"


# Load CSV
train_NB_lem = pd.read_csv(file_path_train)
#val_NB_lem = pd.read_csv(file_path_val)
test_NB_lem = pd.read_csv(file_path_test)

In [ ]:
# Provide the correct file path
file_path_train = "/content/drive/My Drive/BA THESIS/data/train_BERT.csv"
#file_path_val = "/content/drive/My Drive/BA THESIS/data/val_BERT.csv"
file_path_test = "/content/drive/My Drive/BA THESIS/data/test_BERT.csv"


# Load CSV
train_BERT = pd.read_csv(file_path_train)
#val_BERT = pd.read_csv(file_path_val)
test_BERT = pd.read_csv(file_path_test)

## Fitting models

We fit a naive BAyes and a BERT model on each subset of the training data by topic and run a prediciton for test data.

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# function to fit a models on a given subset and then run the classification and return the classification

def NB_on_subset (topic):

  #tarining:
  train_data = train_NB_lem[train_NB_lem['topic'] == topic]
  x = train_data['text']
  y = train_data['ground_truth']

  vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=20000)
  X_train_count = vectorizer.fit_transform(x)

  nb = MultinomialNB(alpha = 1)
  nb.fit(X_train_count, y)

  # testing
  x_test = vectorizer.transform(test_NB_lem['text'])
  y_test = test_NB_lem['ground_truth']
  y_pred = nb.predict(x_test)

  return y_pred

In [ ]:
nb_finance_preds = NB_on_subset("finance")

In [ ]:
nb_news_preds = NB_on_subset("news")

In [ ]:
nb_reddit_pred = NB_on_subset("reddit")

In [ ]:
nb_twitter_preds = NB_on_subset("twitter")

In [ ]:
train_NB_lem.dropna(inplace=True)

In [ ]:
nb_movie_preds = NB_on_subset("movie")

In [ ]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification
from sklearn.metrics import f1_score


In [ ]:
# function to fit a models on a given subset and then run the classification and return the classification

def BERT_on_subset (topic):

  train_data = train_BERT[train_BERT['topic'] == topic]
  test_data = test_BERT.copy()

  # encode labels
  label_encoder = LabelEncoder()
  train_data["label"] = label_encoder.fit_transform(train_data["ground_truth"])
  test_data["label"] = label_encoder.transform(test_data["ground_truth"])

  tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

  def tokenize_function(text):
    return tokenizer(
        text,
        truncation=True,
        padding="longest",    # or use "longest" for dynamic padding
        max_length=200,       # since we are using headlines
        return_tensors="tf"
    )

  # Tokenize texts and extract tensors
  tokenized_train = tokenize_function(train_data["text"].tolist())
  tokenized_test = tokenize_function(test_data["text"].tolist())

  # Convert labels to a TensorFlow tensor
  labels_train = tf.convert_to_tensor(train_data["label"].tolist())
  labels_test = tf.convert_to_tensor(test_data["label"].tolist())

  # Create a dataset from the dictionary and labels
  dataset_train = tf.data.Dataset.from_tensor_slices((
      {
          "input_ids": tokenized_train["input_ids"],
          "attention_mask": tokenized_train["attention_mask"]
      },
      labels_train
  ))

  dataset_test = tf.data.Dataset.from_tensor_slices((
      {
          "input_ids": tokenized_test["input_ids"],
          "attention_mask": tokenized_test["attention_mask"]
      },
      labels_test
  ))

  batch_size = 16
  dataset_train = dataset_train.shuffle(buffer_size=len(train_BERT)).batch(batch_size)

  model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

  # Compile the model with an optimizer, loss, and metrics
  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  num_epochs = 2
  history = model.fit(
      dataset_train,
#      validation_data=dataset_val,
      epochs=num_epochs
  )

  # testing

  logits = model.predict(tokenized_test).logits

  # Convert logits to class predictions
  y_pred_bert = tf.argmax(logits, axis=-1).numpy()

  label_map = {0: "negative", 1: "neutral", 2: "positive"}
  predicted_labels_bert = [label_map[pred] for pred in y_pred_bert]

  return predicted_labels_bert

In [ ]:
bert_finance_preds = BERT_on_subset("finance")

In [ ]:
bert_news_preds = BERT_on_subset("news")

In [ ]:
bert_reddit_preds = BERT_on_subset("reddit")

In [ ]:
bert_twitter_preds = BERT_on_subset("twitter")

In [ ]:
bert_movie_preds = BERT_on_subset("movie")

In [ ]:
# create a data frame with all predictions

all_preds_subsets = pd.DataFrame({
    'Text': test_BERT['text'],
    'Topic': test_NB_lem['topic'],
    'Actual': test_NB_lem['ground_truth'],
    'NB_finance': nb_finance_preds,
    'BERT_finance': bert_finance_preds,
    'NB_news': nb_news_preds,
    'BERT_news': bert_news_preds,
    'NB_reddit': nb_reddit_pred,
    'BERT_reddit': bert_reddit_preds,
    'NB_twitter': nb_twitter_preds,
    'BERT_twitter': bert_twitter_preds,
    'NB_movie': nb_movie_preds,
    'BERT_movie': bert_movie_preds
})


In [ ]:
all_preds_subsets

In [ ]:
all_preds_subsets.to_csv('/content/drive/My Drive/BA THESIS/analysis/all_preds_subsets.csv', index=False)

Now the analysis of performance of these models trained on only a sinle topic.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def calculate_accuracy(predictions, actual):
    correct_predictions = np.sum(np.array(predictions) == np.array(actual))
    return correct_predictions / len(actual)

def create_diamond_chart(ax, model_name, topic_accuracies):
    topics = list(topic_accuracies.keys())
    accuracies = list(topic_accuracies.values())

    angles = np.linspace(0, 2 * np.pi, len(topics), endpoint=False).tolist()
    values = accuracies + [accuracies[0]]
    angles += angles[:1]

    ax.plot(angles, values, linewidth=1, linestyle='solid')
    ax.fill(angles, values, 'skyblue', alpha=0.4)
    ax.set_thetagrids(np.degrees(angles[:-1]), topics)

    ax.set_title(f"{model_name} Accuracy")
    ax.grid(True)


# Calculate accuracies for NB_finance
nb_finance_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic][f"NB_{topic.lower().replace(' ', '_')}"].tolist()
    nb_finance_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)


# Calculate accuracies for BERT_finance
bert_finance_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic][f"BERT_{topic.lower().replace(' ', '_')}"].tolist()
    bert_finance_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)

# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6), subplot_kw={'polar': True})


# Create the diamond chart for NB_finance
create_diamond_chart(ax1, "NB_finance", nb_finance_topic_accuracies)

# Create the diamond chart for BERT_finance
create_diamond_chart(ax2, "BERT_finance", bert_finance_topic_accuracies)

plt.tight_layout()
plt.show()


In [ ]:

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np

def create_diamond_chart(ax, model_name, topic_accuracies, color):
    topics = list(topic_accuracies.keys())
    accuracies = list(topic_accuracies.values())

    angles = np.linspace(0, 2 * np.pi, len(topics), endpoint=False).tolist()
    values = accuracies + [accuracies[0]]
    angles += angles[:1]

    ax.plot(angles, values, linewidth=1, linestyle='solid', color=color)
    ax.fill(angles, values, color, alpha=0.4)
    ax.set_thetagrids(np.degrees(angles[:-1]), topics)
    ax.set_rlim(0,1)

    ax.set_title(f"Performance of model trained on 'finance' data")
    ax.grid(True)


# Assuming all_preds_subsets DataFrame is already available

# Calculate accuracies for NB_finance
nb_finance_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["NB_finance"].tolist()
    nb_finance_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)


# Calculate accuracies for BERT_finance
bert_finance_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["BERT_finance"].tolist()
    bert_finance_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)

# Create the plot
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'polar': True})

# Plot NB_finance data
create_diamond_chart(ax, "NB_finance", nb_finance_topic_accuracies, "blue")

# Plot BERT_finance data
create_diamond_chart(ax, "BERT_finance", bert_finance_topic_accuracies, "green")


# Add legend
legend_elements = [
    Line2D([0], [0], color='blue', lw=2, label='Naive Bayes'),
    Line2D([0], [0], color='green', lw=2, label='BERT')
]
#ax.legend(["Naive Bayes", "BERT"], loc="upper right")
ax.legend(handles=legend_elements, loc="upper right")

plt.tight_layout()
plt.savefig('/content/drive/My Drive/BA THESIS/analysis/performance_finance.png')
plt.show()


In [ ]:
def create_diamond_chart(ax, model_name, topic_accuracies, color):
    topics = list(topic_accuracies.keys())
    accuracies = list(topic_accuracies.values())

    angles = np.linspace(0, 2 * np.pi, len(topics), endpoint=False).tolist()
    values = accuracies + [accuracies[0]]
    angles += angles[:1]

    ax.plot(angles, values, linewidth=1, linestyle='solid', color=color)
    ax.fill(angles, values, color, alpha=0.4)
    ax.set_thetagrids(np.degrees(angles[:-1]), topics)
    ax.set_rlim(0,1)

    ax.set_title(f"Performance of model trained on 'news' data")
    ax.grid(True)


# Assuming all_preds_subsets DataFrame is already available

# Calculate accuracies for NB_news
nb_news_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["NB_news"].tolist()
    nb_news_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)


# Calculate accuracies for BERT_news
bert_news_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["BERT_news"].tolist()
    bert_news_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)

# Create the plot
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'polar': True})

# Plot NB_news data
create_diamond_chart(ax, "NB_news", nb_news_topic_accuracies, "blue")

# Plot BERT_news data
create_diamond_chart(ax, "BERT_news", bert_news_topic_accuracies, "green")


# Add legend
legend_elements = [
    Line2D([0], [0], color='blue', lw=2, label='Naive Bayes'),
    Line2D([0], [0], color='green', lw=2, label='BERT')
]
#ax.legend(["Naive Bayes", "BERT"], loc="upper right")
ax.legend(handles=legend_elements, loc="upper right")

plt.tight_layout()
plt.savefig('/content/drive/My Drive/BA THESIS/analysis/performance_news.png')
plt.show()

In [ ]:
def create_diamond_chart(ax, model_name, topic_accuracies, color):
    topics = list(topic_accuracies.keys())
    accuracies = list(topic_accuracies.values())

    angles = np.linspace(0, 2 * np.pi, len(topics), endpoint=False).tolist()
    values = accuracies + [accuracies[0]]
    angles += angles[:1]

    ax.plot(angles, values, linewidth=1, linestyle='solid', color=color)
    ax.fill(angles, values, color, alpha=0.4)
    ax.set_thetagrids(np.degrees(angles[:-1]), topics)
    ax.set_rlim(0,1)

    ax.set_title(f"Performance of model trained on 'reddit' data")
    ax.grid(True)


# Assuming all_preds_subsets DataFrame is already available

# Calculate accuracies for NB_reddit
nb_reddit_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["NB_reddit"].tolist()
    nb_reddit_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)


# Calculate accuracies for BERT_reddit
bert_reddit_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["BERT_reddit"].tolist()
    bert_reddit_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)

# Create the plot
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'polar': True})

# Plot NB_reddit data
create_diamond_chart(ax, "NB_reddit", nb_reddit_topic_accuracies, "blue")

# Plot BERT_reddit data
create_diamond_chart(ax, "BERT_reddit", bert_reddit_topic_accuracies, "green")


# Add legend
legend_elements = [
    Line2D([0], [0], color='blue', lw=2, label='Naive Bayes'),
    Line2D([0], [0], color='green', lw=2, label='BERT')
]
#ax.legend(["Naive Bayes", "BERT"], loc="upper right")
ax.legend(handles=legend_elements, loc="upper right")

plt.tight_layout()
plt.savefig('/content/drive/My Drive/BA THESIS/analysis/performance_reddit.png')
plt.show()

In [ ]:
def create_diamond_chart(ax, model_name, topic_accuracies, color):
    topics = list(topic_accuracies.keys())
    accuracies = list(topic_accuracies.values())

    angles = np.linspace(0, 2 * np.pi, len(topics), endpoint=False).tolist()
    values = accuracies + [accuracies[0]]
    angles += angles[:1]

    ax.plot(angles, values, linewidth=1, linestyle='solid', color=color)
    ax.fill(angles, values, color, alpha=0.4)
    ax.set_thetagrids(np.degrees(angles[:-1]), topics)
    ax.set_rlim(0,1)

    ax.set_title(f"Performance of model trained on 'twitter' data")
    ax.grid(True)


# Assuming all_preds_subsets DataFrame is already available

# Calculate accuracies for NB_twitter
nb_twitter_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["NB_twitter"].tolist()
    nb_twitter_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)


# Calculate accuracies for BERT_twitter
bert_twitter_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["BERT_twitter"].tolist()
    bert_twitter_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)

# Create the plot
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'polar': True})

# Plot NB_twitter data
create_diamond_chart(ax, "NB_twitter", nb_twitter_topic_accuracies, "blue")

# Plot BERT_twitter data
create_diamond_chart(ax, "BERT_twitter", bert_twitter_topic_accuracies, "green")


# Add legend
legend_elements = [
    Line2D([0], [0], color='blue', lw=2, label='Naive Bayes'),
    Line2D([0], [0], color='green', lw=2, label='BERT')
]
#ax.legend(["Naive Bayes", "BERT"], loc="upper right")
ax.legend(handles=legend_elements, loc="upper right")

plt.tight_layout()
plt.savefig('/content/drive/My Drive/BA THESIS/analysis/performance_twitter.png')
plt.show()

In [ ]:
def create_diamond_chart(ax, model_name, topic_accuracies, color):
    topics = list(topic_accuracies.keys())
    accuracies = list(topic_accuracies.values())

    angles = np.linspace(0, 2 * np.pi, len(topics), endpoint=False).tolist()
    values = accuracies + [accuracies[0]]
    angles += angles[:1]

    ax.plot(angles, values, linewidth=1, linestyle='solid', color=color)
    ax.fill(angles, values, color, alpha=0.4)
    ax.set_thetagrids(np.degrees(angles[:-1]), topics)
    ax.set_rlim(0,1)

    ax.set_title(f"Performance of model trained on 'movie' data")
    ax.grid(True)


# Assuming all_preds_subsets DataFrame is already available

# Calculate accuracies for NB_movie
nb_movie_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["NB_movie"].tolist()
    nb_movie_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)


# Calculate accuracies for BERT_movie
bert_movie_topic_accuracies = {}
for topic in ["finance", "news", "reddit", "twitter", "movie"]:
    actual_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["Actual"].tolist()
    predicted_labels = all_preds_subsets[all_preds_subsets["Topic"] == topic]["BERT_movie"].tolist()
    bert_movie_topic_accuracies[topic] = calculate_accuracy(predicted_labels, actual_labels)

# Create the plot
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'polar': True})

# Plot NB_movie data
create_diamond_chart(ax, "NB_movie", nb_movie_topic_accuracies, "blue")

# Plot BERT_movie data
create_diamond_chart(ax, "BERT_movie", bert_movie_topic_accuracies, "green")


# Add legend
legend_elements = [
    Line2D([0], [0], color='blue', lw=2, label='Naive Bayes'),
    Line2D([0], [0], color='green', lw=2, label='BERT')
]
#ax.legend(["Naive Bayes", "BERT"], loc="upper right")
ax.legend(handles=legend_elements, loc="upper right")

plt.tight_layout()
plt.savefig('/content/drive/My Drive/BA THESIS/analysis/performance_movie.png')
plt.show()